In [1]:
# ref: https://www.kaggle.com/jacklinggu/lstm-with-glove-embedding-public-lb-score-0-049

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
from keras.models import Model, load_model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout,GlobalAveragePooling1D,Conv1D
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

max_features = 40000
maxlen = 150

def clean_text( text ):
    text = text.lower().split()
    text = " ".join(text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+\-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    #
    return text

train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

list_sentences_train = train["comment_text"].fillna("CVxTz").apply(clean_text).values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_test = test["comment_text"].fillna("CVxTz").apply(clean_text).values
print(y.shape)

Using TensorFlow backend.


(159571, 6)


In [2]:
list_sentences_train[:5]

array([ 'explanation why the edits made under my username hardcore metallica fan were reverted they were not vandalisms just closure on some gas after i voted at new york dolls fac and please do not remove the template from the talk page since i am retired now 89 205 38 27',
       'd aww ! he matches this background colour i am seemingly stuck with thanks talk 21 51 january 11 2016 utc ',
       'hey man i am really not trying to edit war it just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page he seems to care more about the formatting than the actual info ',
       ' more i cannot make any real suggestions on improvement - i wondered if the section statistics should be later on or a subsection of types of accidents - i think the references may need tidying so that they are all in the exact same format ie date format etc i can do that later on if no - one else does first - if you have any preferences for formatting styl

In [3]:
print('test len',len(test))

test len 153164


In [4]:
tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)
X_train = sequence.pad_sequences(list_tokenized_train, maxlen=maxlen)
X_test = sequence.pad_sequences(list_tokenized_test, maxlen=maxlen)

print(X_train.shape,X_test.shape)

(159571, 150) (153164, 150)


In [5]:
# check word_index
tmp_cnt = 0
for k in tokenizer.word_index:
    print(k,tokenizer.word_index[k])
    tmp_cnt += 1
    if tmp_cnt >5:
        break
word_idx = tokenizer.word_index

suberb 132478
featural 133420
sendic 98228
rdh 66152
semperfly 159653
optimizer 38896


In [6]:
# read word2vec
# 
word_vec_dict = {}
with open('../glove.840B.300d.txt') as f:
    for line in f:
        v_list = line.strip().split(' ')
        k = str(v_list[0])
        v = np.array([float(x) for x in v_list[1:]])
        word_vec_dict[k] = v
print(len(word_vec_dict))
print('Preparing embedding matrix')
EMBEDDING_DIM = 300
nb_words = min(max_features,len(word_idx))
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word,i in word_idx.items():
    if i >= max_features:
        continue
    else:
        if word in word_vec_dict:
            embedding_matrix[i] = word_vec_dict[word]
print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
del word_vec_dict

2196007
Preparing embedding matrix
Null word embeddings: 3388


In [7]:
from sklearn.metrics import log_loss,accuracy_score

def eval_val(y,train_x):
    res = 0
    acc_res = 0
    for i in range(6):
        curr_loss = log_loss(y[:,i],train_x[:,i])
        acc = accuracy_score(y[:,i],train_x[:,i].round())
        print(i,curr_loss,acc)
        res += curr_loss
        acc_res += acc
    print('final',res/6, acc_res/6)

def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix],trainable=False)(inp)
    x = Dropout(0.2)(x)
    x = Conv1D(256,
             3,
             padding='valid',
             activation='relu',
             strides=1)(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model
print('def model done')


def model done


In [10]:
from sklearn.utils import shuffle

def del_data_ratio(x,y,ratio=0.8):
    pos_index = np.where(y[:,0]==1)[0]
    neg_index = np.where(y[:,0]==0)[0]
    #print(pos_index)
    data_cnt = len(pos_index)
    add_cnt = int(data_cnt*ratio)
    add_index = pos_index[:add_cnt]
    add_x = np.concatenate([x[add_index],x[neg_index]])
    add_y = np.concatenate([y[add_index],y[neg_index]])
    print(x.shape,add_x.shape,data_cnt)
    add_x,add_y = shuffle(add_x,add_y,random_state=666)
    return add_x,add_y

from sklearn.model_selection import KFold
def kf_train(fold_cnt=3,rnd=1):
    kf = KFold(n_splits=fold_cnt, shuffle=False, random_state=233*rnd)
    train_pred, test_pred = np.zeros((159571,6)),np.zeros((153164,6))
    for train_index, test_index in kf.split(X_train):
        # x,y
        curr_x,curr_y = X_train[train_index],y[train_index]
        hold_out_x,hold_out_y = X_train[test_index],y[test_index]
        
        # change pos ratio
        new_curr_x,new_curr_y = del_data_ratio(curr_x,curr_y)
        new_hold_x,new_hold_y = del_data_ratio(hold_out_x,hold_out_y)
        
        # model
        model = get_model()
        batch_size = 64
        epochs = 6
        file_path="weights_base.best.h5"
        checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
        callbacks_list = [checkpoint] 
        
        # train and pred
        model.fit(new_curr_x, new_curr_y, 
                  batch_size=batch_size, epochs=epochs, 
                  validation_data=(new_hold_x,new_hold_y), 
                  callbacks=callbacks_list)
        
        model = load_model(file_path)
        y_test = model.predict(X_test)
        test_pred += y_test
        hold_out_pred = model.predict(hold_out_x)
        train_pred[test_index] = hold_out_pred
    test_pred = test_pred / fold_cnt
    print('-------------------------------')
    print('all eval',eval_val(y,train_pred))
    return train_pred, test_pred


print('def done')

def done


In [11]:
import pickle
sample_submission = pd.read_csv("../input/sample_submission.csv")

for f in [3,4]:
    train_pred,test_pred = kf_train(fold_cnt=f)
    print(train_pred.shape,test_pred.shape)    
    sample_submission[list_classes] = test_pred
    sample_submission.to_csv("../results/cnn1_glove_sample_{}.gz".format(f), index=False, compression='gzip')
    with open('../features/cnn1_glove_sample_feat_{}.pkl'.format(f),'wb') as fout:
        pickle.dump([train_pred,test_pred],fout)
    sample_submission.head()
    print('===================================')


(106380, 150) (104358, 150) 10110
(53191, 150) (52154, 150) 5184
Train on 104358 samples, validate on 52154 samples
Epoch 1/6
104358/104358 [==============================] - 29s 277us/step - loss: 0.0522 - acc: 0.9819 - val_loss: 0.0414 - val_acc: 0.9843
Epoch 2/6
104358/104358 [==============================] - 29s 278us/step - loss: 0.0423 - acc: 0.9845 - val_loss: 0.0393 - val_acc: 0.9850
Epoch 3/6
104358/104358 [==============================] - 29s 275us/step - loss: 0.0393 - acc: 0.9852 - val_loss: 0.0377 - val_acc: 0.9853
Epoch 4/6
104358/104358 [==============================] - 29s 275us/step - loss: 0.0370 - acc: 0.9857 - val_loss: 0.0387 - val_acc: 0.9848
Epoch 5/6
104358/104358 [==============================] - 28s 273us/step - loss: 0.0354 - acc: 0.9860 - val_loss: 0.0380 - val_acc: 0.9856
Epoch 6/6
104358/104358 [==============================] - 29s 278us/step - loss: 0.0338 - acc: 0.9867 - val_loss: 0.0395 - val_acc: 0.9847
(106381, 150) (104330, 150) 10252
(53190, 15

117381/117381 [==============================] - 34s 292us/step - loss: 0.0514 - acc: 0.9821 - val_loss: 0.0411 - val_acc: 0.9847
Epoch 2/6
117381/117381 [==============================] - 34s 291us/step - loss: 0.0417 - acc: 0.9845 - val_loss: 0.0386 - val_acc: 0.9851
Epoch 3/6
117381/117381 [==============================] - 34s 291us/step - loss: 0.0390 - acc: 0.9851 - val_loss: 0.0389 - val_acc: 0.9854
Epoch 4/6
117381/117381 [==============================] - 34s 289us/step - loss: 0.0371 - acc: 0.9856 - val_loss: 0.0394 - val_acc: 0.9850
Epoch 5/6
117381/117381 [==============================] - 34s 290us/step - loss: 0.0353 - acc: 0.9860 - val_loss: 0.0390 - val_acc: 0.9849
Epoch 6/6
117381/117381 [==============================] - 34s 291us/step - loss: 0.0339 - acc: 0.9864 - val_loss: 0.0395 - val_acc: 0.9850
(119678, 150) (117363, 150) 11575
(39893, 150) (39149, 150) 3719
Train on 117363 samples, validate on 39149 samples
Epoch 1/6
117363/117363 [=============================